# Deliverable 3. Create a Travel Itinerary Map.
----
1. Import the vacation CSV file. 
2. From the map above with pick 4 cities that are in close proximity (on the same continent) that the customer would travel to and create a directions layer map.
3. For the "travel_mode" use either "DRIVING", "BICYCLING", or "WALKING".
4. Take a screenshot of the route and save it. 
5. Then create a marker layer for the four cities. Each city will should have a pop-up marker that contains
    - The hotel name
    - The city
    - The country
    - The current weather description and the maximum temperature
---

## Make sure the initial dependencies and the Google API key are imported.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

In [2]:
# Configure gmaps
gmaps.configure(api_key=g_key)

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [3]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../WeatherPy_vacation.csv"
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv("../WeatherPy_vacation.csv")


# Display sample data
vacation_df.head()
# Display sample data
# YOUR CODE HERE

,City_ID,City,Country,Max Temp,Conditions,Lat,Lng,Hotel Name
0,9,Saint George,US,79.43,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
1,12,Cizre,TR,83.95,clear sky,37.3272,42.1903,Divan Cizre Hotel
2,15,Sur,OM,82.80,broken clouds,22.5667,59.5289,Sur Plaza Hotel
3,21,Atuona,PF,76.84,broken clouds,-9.8000,-139.0333,Villa Enata
4,23,Cairo,EG,77.76,clear sky,30.0626,31.2497,Grand Hotel


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [4]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## In this step, you will set-up the pop-up markers. Review the code to create a marker layer map of the vacation search results. This code is the same as in Deliverable 2.

In [5]:
# Review the formatting template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Conditions} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [6]:
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display the figure containing the map
fig


Figure(layout=FigureLayout(height='420px'))

## From the vacation search map, *choose four cities* that a customer might want to visit. They should be close together and in the same country.

Define "close together" to a guy who will ride a motorcycle a thousand miles in a day...

In [7]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# The starting and ending city should be the same city.
vacation_start = vacation_df.loc[vacation_df['City'] == 'Safford']
vacation_end = vacation_df.loc[vacation_df['City'] == 'Safford']
vacation_stop1 = vacation_df.loc[vacation_df['City'] == 'Rancho Palos Verdes']
vacation_stop2 = vacation_df.loc[vacation_df['City'] == 'Half Moon Bay'] 
vacation_stop3 = vacation_df.loc[vacation_df['City'] == 'Saint George'] 
vacation_start

,City_ID,City,Country,Max Temp,Conditions,Lat,Lng,Hotel Name
214,662,Safford,US,75.11,clear sky,32.834,-109.7076,SpringHill Suites by Marriott Thatcher


## Use the `to_numpy()` function and list indexing to write code to retrieve the latitude-longitude pairs as tuples from each city DataFrame.

In [8]:
# Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start.iloc[0,5:7].to_numpy()
end = vacation_end.iloc[0,5:7].to_numpy()
stop1 = vacation_stop1.iloc[0,5:7].to_numpy()
stop2 = vacation_stop2.iloc[0,5:7].to_numpy()
stop3 = vacation_stop3.iloc[0,5:7].to_numpy()

start

array([32.834, -109.7076], dtype=object)

In [11]:
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat(
    [
        vacation_start,
        vacation_stop1,
        vacation_stop2,
        vacation_stop3
    ],
    ignore_index = True
)

# Display sample data
itinerary_df

,City_ID,City,Country,Max Temp,Conditions,Lat,Lng,Hotel Name
0,662,Safford,US,75.11,clear sky,32.8340,-109.7076,SpringHill Suites by Marriott Thatcher
1,469,Rancho Palos Verdes,US,78.08,broken clouds,33.7445,-118.3870,Terranea Resort
2,95,Half Moon Bay,US,79.88,mist,37.4636,-122.4286,"The Ritz-Carlton, Half Moon Bay"
3,9,Saint George,US,79.43,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn


## Use the [gmaps documentation](https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#directions-layer) to create a directions layer map using the variables from the previous step. Where the starting and ending city are the same city, the `waypoints` are the three other cities, and the `travel_mode` is either `"DRIVING"`, `"BICYCLING"`, or `"WALKING"`.

In [12]:
# Define a new figure object
fig = gmaps.figure(center=(35, -125), zoom_level = 7)
Safford = start
Rancho_Palos_Verdes = stop1
Half_Moon_Bay = stop2
Saint_George = stop3 
roadtrip = gmaps.directions_layer(Safford, Safford, 
                                  waypoints=[Rancho_Palos_Verdes, Half_Moon_Bay, Saint_George],
                                  travel_mode="DRIVING")

fig.add_layer(roadtrip)                                  

# Create a direction layer map using the start and end latitude-longitude pairs, and stop1, stop2, and stop3 as the waypoints.
# The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Conditions} and {Max Temp} °F</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]
# Add a marker layer for each city to the map.
fig = gmaps.figure(center=(35.0, -116.0), zoom_level=5,layout={'width': '1400px', 'height': '1000px'})
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(roadtrip)

# Display the figure
fig 



Figure(layout=FigureLayout(height='1000px', width='1400px'))

## Use the provided `concat()` function code snippet to combine the four separate city DataFrames into one DataFrame.

## Refactor the code from Step 6 to create a new marker layer map of the cities on the travel route.

In [ ]:
#Review the formatting template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Conditions} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

hotel_info
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [ ]:
print(locations)

In [ ]:
# Add a marker layer for each city to the map.
fig = gmaps.figure(center=(35.0, -116.0), zoom_level=6,layout={'width': '1400px', 'height': '1000px'})

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display the figure
fig 